In [1]:
# TBD 1 : logger 추가
# TBD 2: flask github 참고, method, class, 파일의 맨 윗단 마다 pydoc 형식으로 달기
# TBD 3: 축약어를 자제할것 (특히 변수)

# -------------------------
#   done
# -------------------------

# 0. add data-setter, receiver system use python queue.Queue() class
# this will resolve i/o bottleneck
# 3. make iterable

# -------------------------
#   In Progress
# -------------------------

# 1. add logger
# 2. make image drawer overlay mask on image

# -------------------------
#   To be Done
# -------------------------

# 4. make verbose turn on and off
# 5. write pydoc

# python basic Module
import os
import sys
import types
import progressbar
from datetime import datetime
from shutil import copy
from pickle import dump, load

# math, image, plot Module
import numpy as np
import cv2
import matplotlib.pyplot as plt  # TBD

# tensorflow Module
import tensorflow as tf
from tensorflow.keras import backend as keras_backend
from tensorflow.keras.layers import GaussianNoise
from tensorflow.keras.layers import Input, Concatenate
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Nadam
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras import losses

# keras segmentaion third-party Moudle
import segmentation_models as sm
import tensorflow_addons as tfa

# custom Module
from gan_module.data_loader.medical_segmentation_data_loader import DataLoader
from gan_module.data_loader.manage_batch import BatchQueueManager

from gan_module.model.build_model import build_generator as build_generator
from gan_module.model.build_model import build_discriminator as build_discriminator
from gan_module.util.custom_loss import weighted_region_loss, dice_score, combined_loss, f1_loss
# from gan_module.util.custom_gradient import SGD_AGC
from gan_module.util.manage_learning_rate import learning_rate_scheduler
from gan_module.util.draw_images import ImageDrawer
from gan_module.util.logger import TrainLogger
from gan_module.config import CONFIG

USE_GPU = True

if USE_GPU:
    os.environ['CUDA_VISIBLE_DEVICES'] = '0'
    gpu_devices = tf.config.experimental.list_physical_devices("GPU")
    for device in gpu_devices:
        tf.config.experimental.set_memory_growth(device, True)
else:
    os.environ['CUDA_VISIBLE_DEVICES'] = '-1'


class Pix2PixSegmentation:
    def __init__(
        self,
        generator_power=32,
        discriminator_power=32,
        generator_depth = None,
        discriminator_depth = None,
        generator_learning_rate=1e-4,
        discriminator_learning_rate=1e-4,
        temp_weights_path=".",
        on_memory=True,
        code_test=False
    ):
        # Input shape
        img_shape = CONFIG["img_shape"]
        input_channels = CONFIG["input_channels"]
        output_channels = CONFIG["output_channels"]

        self.input_img_shape = (*img_shape, input_channels)
        self.output_img_shape = (*img_shape, output_channels)
        # set parameter
        self.start_epoch = None
        self.on_memory = on_memory
        self.history = {"generator_loss": [],
                        "f1_loss_train": [], "f1_score_train": [],
                        "f1_loss_valid": [], "f1_score_valid": []}
        self.temp_weights_path = temp_weights_path

        # Configure data loader
        self.dataset_name = "glomerulus_0.65_512_remove_peel_split_man"
        self.data_loader = DataLoader(
            dataset_name=self.dataset_name,
            config_dict=CONFIG,
            on_memory=self.on_memory, 
            code_test=code_test
        )
        
        self.train_logger = TrainLogger()
        
        self.loaded_data_index = {
            "train": np.arange(self.data_loader.data_length["train"]),
            "valid": np.arange(self.data_loader.data_length["valid"])
        }
        
        # Configure Image Drawer
        self.image_drawer = ImageDrawer(
            dataset_name=self.dataset_name, data_loader=self.data_loader
        )
        self.discriminator_loss_ratio = keras_backend.variable(0.1)
        self.f1_loss_ratio = keras_backend.variable(100)
        self.discriminator_losses = np.array(
            [1 for _ in range(self.data_loader.data_length["train"])], dtype=np.float32)
        self.discriminator_acc_previous = 0.5
        self.discriminator_acces = np.array(
            [0.5 for _ in range(self.data_loader.data_length["train"])])
        self.discriminator_acces_previous = self.discriminator_acces.copy()
        self.generator_losses = np.array(
            [1 for _ in range(self.data_loader.data_length["train"])], dtype=np.float32)
        self.generator_losses_previous = self.generator_losses.copy()
        self.generator_f1_losses = np.array(
            [1 for _ in range(self.data_loader.data_length["train"])], dtype=np.float32)
        self.generator_loss_min = 10000
        self.generator_loss_previous = 10000
        self.generator_loss_max_previous = 10000
        self.generator_valid_loss_min = 10000
        self.total_f1_loss_min = 10000
        self.weight_save_stack = False
        self.training_end_stack = 0
        # Calculate output shape of D (PatchGAN)
        self.disc_patch = (img_shape[0] // (2 ** discriminator_depth), img_shape[1] // (2 ** discriminator_depth), 1)
        # Number of filters in the first layer of G and D
        self.generator_learning_rate = generator_learning_rate
        self.discriminator_learning_rate = discriminator_learning_rate
        self.patience_count = 0
        
        generator_optimizer = Nadam(self.generator_learning_rate)
        discriminator_optimizer = Nadam(self.discriminator_learning_rate)
#         generator_optimizer = SGD_AGC(lr=self.generator_learning_rate, momentum=0.9)
#         discriminator_optimizer = SGD_AGC(lr=self.discriminator_learning_rate, momentum=0.9)        
        # Build the generator
        self.generator = build_generator(
            input_img_shape=self.input_img_shape,
            output_channels=output_channels,
            generator_power=generator_power,
            depth=generator_depth,
        )
        self.generator.compile(
            loss=weighted_region_loss,
            optimizer=generator_optimizer,
            metrics=[dice_score],
        )
        # Build and compile the discriminator
        self.discriminator = build_discriminator(
            input_img_shape=self.input_img_shape,
            output_img_shape=self.output_img_shape,
            discriminator_power=discriminator_power,
            depth=discriminator_depth,
        )
        # 'mse' or tf.keras.losses.Huber() tf.keras.losses.LogCosh()
        self.discriminator.compile(
            loss=sm.losses.BinaryFocalLoss(alpha=0.25, gamma=4),
            optimizer=discriminator_optimizer,
            metrics=["accuracy"],
        )
        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # -------------------------
        # Construct Computational
        #   Graph of Generator
        # -------------------------

        # Input images and their conditioning images
        original_img = Input(shape=self.input_img_shape)
        # generate image from original_img for target masked_img
        model_masked_img = self.generator(original_img)
        
    
        # Discriminators determines validity of translated images / condition pairs
        model_validity = self.discriminator([original_img, model_masked_img])
        # give score by
        # 1. how generator trick discriminator
        # 2. how generator's image same as real photo in pixel
        # 3. if you want change loss, see doc https://keras.io/api/losses/
        # 4. 'mse', 'mae', tf.keras.losses.LogCosh(),  tf.keras.losses.Huber()
        self.combined = Model(
            inputs=original_img,
            outputs=[model_validity, model_masked_img],
        )
        
        self.combined.compile(
            loss=[
#                 tf.keras.losses.BinaryCrossentropy(label_smoothing=0.05),
                sm.losses.BinaryFocalLoss(alpha=0.25, gamma=4),
                weighted_region_loss
            ],
            loss_weights=[0.1, 100],
            optimizer=generator_optimizer,
        )

    def train(self, epochs, batch_size=1, epoch_shuffle_term=10):

        start_time = datetime.now()

        # Adversarial loss ground truths
        self.training_end_stack = 0
        self.batch_size = batch_size
        valid_patch = np.ones(
            (self.batch_size, *self.disc_patch), dtype=np.float32)
        fake_patch = np.zeros(
            (self.batch_size, *self.disc_patch), dtype=np.float32)
        # TBD : move batch_queue_manager to __init__
        self.batch_queue_manager = BatchQueueManager(self, batch_size, self.on_memory)
        
        if self.start_epoch is None:
            self.start_epoch = 0
        for epoch in range(self.start_epoch, epochs):
            batch_i = 0

            generator_loss_max_in_epoch = 0
            generator_loss_min_in_epoch = 1000
            generator_discriminator_losses = np.array(
            [1 for _ in range(self.data_loader.data_length["train"])], dtype=np.float32)
            # shffle data maybe
            if epoch % epoch_shuffle_term == 0:
                self.data_loader.shuffle_train_imgs()
            
            if self.discriminator_acc_previous < 0.95:
                discriminator_learning = True
            else:
                discriminator_learning = False
            generator_1_10_quantile = np.quantile(self.generator_losses, 0.1)
            
            
            generator_current_learning_rate = learning_rate_scheduler(
                self.generator_learning_rate,
                epoch+self.patience_count,
                warm_up=True
            )
            discriminator_current_learning_rate = learning_rate_scheduler(
                self.discriminator_learning_rate,
                epoch+self.patience_count,
                warm_up=True
            ) * (1 - self.discriminator_acc_previous)
            keras_backend.set_value(
                self.discriminator.optimizer.learning_rate,
                discriminator_current_learning_rate,
            )
            keras_backend.set_value(
                self.discriminator.optimizer.learning_rate,
                discriminator_current_learning_rate,
            )
            keras_backend.set_value(
                self.discriminator_loss_ratio,
                keras_backend.variable(0.01) + 0.375 * self.discriminator_acc_previous,
            )
            keras_backend.set_value(
                self.f1_loss_ratio,
                keras_backend.variable(100) - 0.375  * self.discriminator_acc_previous,
            )
            
            bar = progressbar.ProgressBar(
                maxval=self.data_loader.data_length["train"]).start()
            
            while batch_i + self.batch_size < self.data_loader.data_length["train"] + self.batch_size:

                batch_index = self.loaded_data_index["train"][batch_i: batch_i +
                                                              self.batch_size]

                original_img, masked_img = self.batch_queue_manager.get_batch(
                    data_mode="train")
                model_masked_img = self.generator.predict_on_batch(
                    original_img)
                
                valid_patch = np.ones(
                    (len(model_masked_img), *self.disc_patch), dtype=np.float32)
                fake_patch = np.zeros(
                    (len(model_masked_img), *self.disc_patch), dtype=np.float32)
                
                self.original_img = original_img
                self.masked_img = masked_img
                # ---------------------
                #  Train Discriminator
                # ---------------------
                # Train Discriminator for valid image if it failed to detect fake image
                if discriminator_learning and self.discriminator_acc_previous < np.random.rand():
                    discriminator_loss = self.discriminator.train_on_batch([original_img, masked_img], valid_patch)
                else:
                    discriminator_loss = self.discriminator.test_on_batch([original_img, masked_img], valid_patch)
                    
                batch_discriminator_acc_previous = np.mean(
                    self.discriminator_acces_previous[batch_index])
                self.discriminator.trainable = False
                # -----------------
                #  Train Generator
                # -----------------
                
#                 if np.mean(self.generator_losses[batch_index]) >= generator_1_10_quantile:
#                     generator_loss = self.combined.train_on_batch(
#                         original_img,
#                         [valid_patch, masked_img]
#                     )
#                 else:
#                     generator_loss = self.combined.test_on_batch(
#                         original_img,
#                         [valid_patch, masked_img]
#                     )
                generator_loss = self.combined.train_on_batch(
                    original_img,
                    [valid_patch, masked_img]     
                )
                # train discriminator for fake image if it failed to detect fake image
                self.discriminator.trainable = True
                if (batch_discriminator_acc_previous <= 0.5 or epoch == 0) and discriminator_learning:
                    discriminator_loss += self.discriminator.train_on_batch(
                        [original_img,model_masked_img], fake_patch)
                else:
                    discriminator_loss += self.discriminator.test_on_batch(
                        [original_img,model_masked_img], fake_patch)

                self.discriminator_losses[batch_index] = discriminator_loss[0]
                self.discriminator_acces[batch_index] = discriminator_loss[1]
                self.generator_losses[batch_index] = generator_loss[0]
                self.generator_f1_losses[batch_index] = generator_loss[2]
                generator_discriminator_losses = generator_loss[1]
                # plot progress
                bar.update(batch_i)
                
                # 한 배치 끝
                batch_i += self.batch_size
            
            # training batch 사이클 끝
            
            #######################################
            # valid_loss, valid score 계산
            #######################################
            
            #if self.generator_loss_min > np.mean(self.generator_losses):
            valid_f1_loss_list = []
            valid_f1_score_list = []
            for index in range(0, self.data_loader.data_length["valid"], self.batch_size):

                valid_source_img, valid_masked_img = self.batch_queue_manager.get_batch(
                    data_mode="valid")

                valid_model_masked_img = self.generator.predict_on_batch(
                    valid_source_img)
                valid_f1_loss =  weighted_region_loss(valid_masked_img, valid_model_masked_img)
                valid_f1_score = dice_score(valid_masked_img, valid_model_masked_img)
                
                valid_f1_loss_list.append(valid_f1_loss)
                valid_f1_score_list.append(valid_f1_score)
            
            current_valid_f1_loss = np.mean(valid_f1_loss_list)
            
            # compute valid_f1_loss end    
            total_f1_loss = np.mean(self.generator_f1_losses) * 0.75 + current_valid_f1_loss * 0.25
                
            self.discriminator_acc_previous = np.mean(self.discriminator_acces)
            self.discriminator_acces_previous = self.discriminator_acces.copy()
            self.generator_losses_previous = self.generator_losses.copy()
            # TBD: add epoch bigger than history length
            self.history["generator_loss"].append(
                np.mean(self.generator_losses))
            self.history["f1_loss_train"].append(
                np.mean(self.generator_f1_losses))
            self.history["f1_loss_valid"].append(
                np.mean(valid_f1_loss_list))
            

            self.image_drawer.sample_images(
                self.generator, epoch)

            # previous generator_loss 갱신
            self.generator_loss_previous = np.mean(self.generator_losses)
            self.generator_loss_max_previous = np.max(self.generator_losses)
            generator_loss_decrease_ratio = np.mean(self.generator_losses) / self.generator_loss_min
            
            #######################################
            # 학습 상태 및 로그 출력
            #######################################
            self.train_logger.write_log(
                f"{epoch}/{epochs} ({epoch+self.patience_count})",
                np.mean(self.discriminator_acces),
                np.mean(self.generator_losses),
                np.max(self.generator_losses),
                np.min(self.generator_losses),
                f"{self.generator_loss_min - np.mean(self.generator_losses)}({generator_loss_decrease_ratio})",
                self.generator_loss_min,
                generator_current_learning_rate,
                datetime.now() - start_time
            )
            print(f"valid_loss : {self.generator_valid_loss_min} / {current_valid_f1_loss}")
            print(f"discriminator_loss : {np.mean(self.discriminator_losses)}")
            print(f"generator_discriminator_loss : {np.mean(generator_discriminator_losses)}")
            print(f"train_f1_loss : {np.mean(self.generator_f1_losses)}")
            print(f"valid_f1_score : {np.mean(valid_f1_score_list)}")
            print(f"current/min total_f1_loss = {self.total_f1_loss_min} / {total_f1_loss}")
            #######################################
            # 학습 상태 관찰 후 저장 여부 선택
            #######################################
            
            if total_f1_loss / self.total_f1_loss_min < 1.1:
                
                if self.total_f1_loss_min > total_f1_loss:
        
                    self.generator_valid_loss_min = current_valid_f1_loss
                    self.total_f1_loss_min = total_f1_loss
                    if self.generator_loss_min > np.mean(self.generator_losses):
                        self.generator_loss_min = np.mean(self.generator_losses)
                        self.generator_loss_max_min = generator_loss_max_in_epoch
                        self.generator_loss_min_min = generator_loss_min_in_epoch
                    
                    self.save_study_info()
                    self.weight_save_stack = True
                    print("save weights")    
                    if epoch < 20:
                        if generator_loss_decrease_ratio < 0.8 and epoch > 0:
                            self.patience_count += 0.25
                    else:
                        self.generator_learning_rate *= generator_loss_decrease_ratio
                        self.discriminator_learning_rate  *= generator_loss_decrease_ratio
                        
                if self.generator_valid_loss_min > current_valid_f1_loss: 
                    self.generator_valid_loss_min = current_valid_f1_loss
                    
            else:
                print("loss decrease.")
                if epoch+self.patience_count < 20:
                    self.patience_count -= 1.5
                else:
                    self.patience_count += 1
                self.load_best_weights()
            
            if epoch >= 10 and self.weight_save_stack:
                copy(
                    "generator.h5",
                    "./generator_weights/generator_"
                    + str(round(self.generator_loss_min, 5))
                    + "_"
                    + str(round(self.generator_loss_max_min, 5))
                    + ".h5",
                )
                self.weight_save_stack = False
            
            # 한 epoch의 끝
        
            
    def get_info_folderPath(self):
        return (
            str(round(self.generator_loss_min, 5))
            + "_"
            + str(round(self.generator_loss_max_min, 5))
        )

    def save_study_info(self, path=None):

        if path is None:
            path = self.temp_weights_path

        generator_weigth_path = os.path.join(path, "generator.h5")
        discriminator_weigth_path = os.path.join(path, "discriminator.h5")
        combined_weigth_path = os.path.join(path, "combined.h5")

        self.generator.save_weights(generator_weigth_path)
        self.discriminator.save_weights(discriminator_weigth_path)
        self.combined.save_weights(combined_weigth_path)

        study_info = {}
        study_info["start_epoch"] = self.start_epoch
        study_info["train_loaded_data_index"] = self.loaded_data_index["train"]
        study_info["generator_loss_min"] = self.generator_loss_min
        study_info["generator_loss_max_min"] = self.generator_loss_max_min
        study_info["generator_loss_min_min"] = self.generator_loss_min_min
        study_info["generator_losses_previous"] = self.generator_losses_previous
        study_info["discriminator_acces"] = self.discriminator_acces
        study_info["history"] = self.history
        file = open(path + "/study_info.pkl", "wb")
        dump(study_info, file)
        file.close()

    def load_study_info(self):

        self.generator.load_weights("generator.h5")
        self.discriminator.load_weights("discriminator.h5")
#         self.combined.load_weights("combined.h5")

        if os.path.isfile("study_info.pkl"):
            file = open("study_info.pkl", "rb")
            study_info = load(file)
            file.close()
            self.start_epoch = study_info["start_epoch"]
            self.loaded_data_index["train"] = study_info["train_loaded_data_index"]
            self.generator_loss_min = study_info["generator_loss_min"]
            self.generator_loss_max_min = study_info["generator_loss_max_min"]
            self.generator_loss_min_min = study_info["generator_loss_min_min"]
            self.generator_losses_previous = study_info["generator_losses_previous"]
            self.discriminator_acces = study_info["discriminator_acces"]
            self.history = study_info["history"]
        else:
            print("No info pkl file!")

    def load_best_weights(self):
        self.generator.load_weights(self.temp_weights_path + "/generator.h5")
        self.discriminator.load_weights(
            self.temp_weights_path + "/discriminator.h5")
        self.combined.load_weights(self.temp_weights_path + "/combined.h5")

    def run_pretraining(self, epochs):
        if self.on_memory:
            self.generator.fit(
                x=self.data_loader.loaded_data_object["train"]["input"],
                y=self.data_loader.loaded_data_object["train"]["output"],
                validation_data=list(self.data_loader.loaded_data_object["valid"].values()),
                batch_size=self.batch_size, epochs=epochs
            )
        else:
            self.generator.fit_generator(
                x=self.data_loader.loaded_data_object["train"]["input"],
                y=self.data_loader.loaded_data_object["train"]["output"],
                validation_data=list(self.data_loader.loaded_data_object["valid"].values()),
                batch_size=self.batch_size, epochs=epochs
            )
        self.generator.save_weights("pretrained.h5")

Segmentation Models: using `tf.keras` framework.
{'img_shape': [512, 512], 'input_channels': 3, 'output_channels': 1}


In [2]:
generator_lr = 1e-5
discriminator_lr = 1e-5
batch_size = 4

g_lr = generator_lr * batch_size
d_lr = discriminator_lr * batch_size
gan = Pix2PixSegmentation(generator_power=8, discriminator_power=8, 
                          generator_depth = 4, discriminator_depth = 4,
                          generator_learning_rate=g_lr, discriminator_learning_rate=d_lr,
                          on_memory=False, code_test=False)

In [ ]:
# gan.load_study_info()
# gan.start_epoch = 0
gan.train(epochs=100, batch_size=batch_size, epoch_shuffle_term=10)

 99% (4496 of 4500) |################### | Elapsed Time: 0:14:11 ETA:   0:00:00{
2021-06-02 06:42:55,149 - train - INFO - 
Epoch : 0/100 (0)
Discriminator_acces : 0.5293157552083333
Mean generator loss : 276.2706604003906
Max generator loss : 323.77618408203125
Min generator loss : 206.41571044921875
Generator loss decrease : 9723.72933959961(0.02762706604003906)
Current lowest generator loss : 10000
Current Learning_rate : 2.0000000000000003e-06
Elapsed_time : 0:15:47.693082
}


valid_loss : 10000 / 2.468461513519287
discriminator_loss : 5.583191871643066
generator_discriminator_loss : 0.01914921961724758
train_f1_loss : 2.4725801944732666
valid_f1_score : 0.18706108629703522
current/min total_f1_loss = 10000 / 2.4715505242347717


N/A% (0 of 4500) |                       | Elapsed Time: 0:00:00 ETA:  --:--:--

save weights


 99% (4496 of 4500) |################### | Elapsed Time: 0:12:21 ETA:   0:00:00{
2021-06-02 06:56:51,881 - train - INFO - 
Epoch : 1/100 (1)
Discriminator_acces : 0.56564453125
Mean generator loss : 265.6551208496094
Max generator loss : 302.1759033203125
Min generator loss : 200.34075927734375
Generator loss decrease : 10.61553955078125(0.9615755677223206)
Current lowest generator loss : 276.2706604003906
Current Learning_rate : 4.000000000000001e-06
Elapsed_time : 0:29:44.424872
}


...

 99% (4496 of 4500) |################### | Elapsed Time: 0:11:58 ETA:   0:00:00{
2021-06-02 07:10:25,517 - train - INFO - 
Epoch : 2/100 (2)
Discriminator_acces : 0.6105088975694445
Mean generator loss : 261.95989990234375
Max generator loss : 298.119140625
Min generator loss : 196.1549072265625
Generator loss decrease : 3.695220947265625(0.9860901832580566)
Current lowest generator loss : 265.6551208496094
Current Learning_rate : 6e-06
Elapsed_time : 0:43:18.060582
}


valid_loss : 2.4249823093414307 / 2.4167356491088867
discriminator_loss : 5.21197509765625
generator_discriminator_loss : 0.021239474415779114
train_f1_loss : 2.3666515350341797
valid_f1_score : 0.19286130368709564
current/min total_f1_loss = 2.398231565952301 / 2.3791725635528564


N/A% (0 of 4500) |                       | Elapsed Time: 0:00:00 ETA:  --:--:--

save weights


 78% (3544 of 4500) |###############     | Elapsed Time: 0:09:09 ETA:   0:02:28

In [ ]:
# gan.load_study_info()
# gan.start_epoch = 0
gan.train(epochs=100, batch_size=batch_size, epoch_shuffle_term=10)

In [4]:
# gan.load_study_info()
# gan.start_epoch = 0
gan.train(epochs=100, batch_size=batch_size, epoch_shuffle_term=10)

 99% (6196 of 6200) |################### | Elapsed Time: 0:18:20 ETA:   0:00:00{
2021-05-30 08:26:31,268 - train - INFO - 
Epoch : 0/100 (0)
Discriminator_acces : 0.46689488564768145
Mean generator loss : 77.13441467285156
Max generator loss : 159.24464416503906
Min generator loss : 61.41813278198242
Generator loss decrease : 9922.865585327148(0.0077134414672851564)
Current lowest generator loss : 10000
Current Learning_rate : 2e-05
Elapsed_time : 0:18:35.350937
}


valid_loss : 10000 / 0.5948840975761414
discriminator_loss : 1.515385389328003
generator_discriminator_loss : 0.008397523313760757
train_f1_loss : 0.7139192223548889
valid_f1_score : 0.17828789353370667
current/min total_f1_loss = 2 / 0.684160441160202


N/A% (0 of 6200) |                       | Elapsed Time: 0:00:00 ETA:  --:--:--

save weights


 99% (6196 of 6200) |################### | Elapsed Time: 0:17:14 ETA:   0:00:00{
2021-05-30 08:43:58,193 - train - INFO - 
Epoch : 1/100 (0.5)
Discriminator_acces : 0.4225358729208669
Mean generator loss : 60.59644317626953
Max generator loss : 152.40994262695312
Min generator loss : 45.45588302612305
Generator loss decrease : 16.53797149658203(0.7855954170227051)
Current lowest generator loss : 77.13441467285156
Current Learning_rate : 3e-05
Elapsed_time : 0:36:02.275368
}


...

valid_loss : 0.18333272635936737 / 0.1850264072418213
discriminator_loss : 0.0200507715344429
generator_discriminator_loss : 0.020054887980222702
train_f1_loss : 0.14980103075504303
valid_f1_score : 0.5240164399147034
current/min total_f1_loss = 0.16111133992671967 / 0.1586073748767376


N/A% (0 of 6200) |                       | Elapsed Time: 0:00:00 ETA:  --:--:--

save weights


 11% (720 of 6200) |##                   | Elapsed Time: 0:01:38 ETA:   0:12:38

KeyboardInterrupt: 

In [3]:
#gan.load_study_info()
gan.start_epoch = 0
gan.train(epochs=40, batch_size=batch_size, epoch_shuffle_term=10)

 99% (6196 of 6200) |################### | Elapsed Time: 0:19:24 ETA:   0:00:00{
2021-05-29 13:01:32,972 - train - INFO - 
Epoch : 0/40 (0)
Discriminator_acces : 0.6410353137600806
Mean generator loss : 75.12113952636719
Max generator loss : 161.28274536132812
Min generator loss : 53.76152420043945
Generator loss decrease : 9924.878860473633(0.007512113952636719)
Current lowest generator loss : 10000
Current Learning_rate : 2e-05
Elapsed_time : 0:19:39.518632
}


valid_loss : 10000 / 0.582449197769165
discriminator_loss : 4.612209320068359
generator_discriminator_loss : 0.008341144770383835
train_f1_loss : 0.615655779838562
valid_f1_score : 0.3262079954147339
current/min total_f1_loss = 2 / 0.6073541343212128


N/A% (0 of 6200) |                       | Elapsed Time: 0:00:00 ETA:  --:--:--

save weights


 99% (6196 of 6200) |################### | Elapsed Time: 0:15:52 ETA:   0:00:00{
2021-05-29 13:17:38,892 - train - INFO - 
Epoch : 1/40 (0.5)
Discriminator_acces : 0.7510049143145161
Mean generator loss : 56.85160446166992
Max generator loss : 141.6635284423828
Min generator loss : 40.799049377441406
Generator loss decrease : 18.269535064697266(0.756799042224884)
Current lowest generator loss : 75.12113952636719
Current Learning_rate : 3e-05
Elapsed_time : 0:35:45.437878
}


...

save weights


 99% (6196 of 6200) |################### | Elapsed Time: 0:15:00 ETA:   0:00:00{
2021-05-29 22:40:59,321 - train - INFO - 
Epoch : 38/40 (7.5)
Discriminator_acces : 0.8634447454637096
Mean generator loss : 13.364004135131836
Max generator loss : 68.0072021484375
Min generator loss : 2.5207767486572266
Generator loss decrease : 0.32065677642822266(0.9765681624412537)
Current lowest generator loss : 13.684660911560059
Current Learning_rate : 0.00017
Elapsed_time : 9:59:05.867219
}


valid_loss : 0.23739232122898102 / 0.30632483959198
discriminator_loss : 0.28013521432876587
generator_discriminator_loss : 0.006585691589862108
train_f1_loss : 0.11857323348522186
valid_f1_score : 0.4711536765098572
current/min total_f1_loss = 0.15020129457116127 / 0.1655111350119114
loss decrease.


 99% (6196 of 6200) |################### | Elapsed Time: 0:14:46 ETA:   0:00:00{
2021-05-29 22:55:59,578 - train - INFO - 
Epoch : 39/40 (7.0)
Discriminator_acces : 0.8661129662298387
Mean generator loss : 13.58354377746582
Max generator loss : 63.548919677734375
Min generator loss : 2.3479647636413574
Generator loss decrease : 0.10111713409423828(0.9926109313964844)
Current lowest generator loss : 13.684660911560059
Current Learning_rate : 0.00016
Elapsed_time : 10:14:06.124040
}


valid_loss : 0.23739232122898102 / 0.23762275278568268
discriminator_loss : 0.281535804271698
generator_discriminator_loss : 0.0055017611011862755
train_f1_loss : 0.1204899325966835
valid_f1_score : 0.5402778387069702
current/min total_f1_loss = 0.15020129457116127 / 0.1497731376439333
save weights


In [ ]:
# gan.load_study_info()
# gan.start_epoch = 8
gan.train(epochs=200, batch_size=batch_size, epoch_shuffle_term=50)
# gan.train(epochs=20, batch_size=batch_size, epoch_shuffle_term=100)

In [ ]:
gan.generator_losses[-4:]

In [ ]:
gan.generator_losses

In [ ]:
valid_patch = np.ones(
    (gan.batch_size, *gan.disc_patch), dtype=np.float32)

generator_loss = gan.combined.train_on_batch(
    gan.original_img,
    [valid_patch, gan.masked_img],
    class_weight={0: 0.1, 1: 0.9}
)

In [ ]:
batch_i=0
batch_index = gan.loaded_data_index["train"][batch_i: batch_i +
                                              gan.batch_size]

In [ ]:
np.mean(gan.generator_losses[batch_index]) > np.quantile(gan.generator_losses, 0.1)

In [ ]:
# gan.load_study_info()
#gan.start_epoch = 30
# gan.start_epoch = 2 
gan.train(epochs=325, batch_size=batch_size, epoch_shuffle_term=100)
# gan.train(epochs=20, batch_size=batch_size, epoch_shuffle_term=100)

In [ ]:
temp_weight = gan.generator.weights

In [ ]:
for weight in temp_weight:
    if "dense" in weight.name:
        if "bias" in weight.name:
            print(weight.name)
            print(weight.numpy())


In [ ]:
temp_weight = gan.discriminator.weights

In [ ]:
for weight in temp_weight:
    if "dense" in weight.name:
        if "bias" in weight.name:
            print(weight.name)
            print(weight.numpy())


In [ ]:
gan.original_img

In [ ]:
np.max(gan.original_img.numpy())

In [ ]:
np.min(gan.original_img.numpy())

In [ ]:
(gan.original_img.numpy() + 1) * 127.5

In [ ]:
image = gan.original_img.numpy()
predicted = gan.generator.predict_on_batch(image)
mask = gan.masked_img.numpy()

image = ((image + 1) * 127.5).astype('uint8')
predicted = ((predicted + 1) * 127.5).astype('uint8')
mask = ((mask + 1) * 127.5).astype('uint8')

In [ ]:
index = 0

plt.figure(figsize=(15, 15))

plt.subplot(131)
plt.imshow(image[index])

plt.subplot(132)
plt.imshow(predicted[index])

plt.subplot(133)
plt.imshow(mask[index])


In [ ]:
print(np.max(gan.original_img))
print(np.min(gan.original_img))
print(np.max(temp))
print(np.min(temp))

In [ ]:
temp = gan.data_loader.loaded_data_object["train"].values()

for index, (input_img, output_img) in enumerate(zip(*temp)):
    print(index)
    if index > 40:
        break
    print(index)
    print(input_img.shape)
    print(output_img.shape)
    

In [ ]:
import time

temp_source = gan.original_img
temp_mask = gan.masked_img

start_time = time.time()
gan.generator.train_on_batch(temp_source, temp_mask)
print(f"elapsed time : {time.time() - start_time}")

temp_source = tf.convert_to_tensor(temp_source)
temp_mask = tf.convert_to_tensor(temp_mask)

start_time = time.time()
gan.generator.train_on_batch(temp_source, temp_mask)
print(f"elapsed time : {time.time() - start_time}")

# Iterator : 260초
# Queue Iterator : 200초

In [ ]:
import time
import threading
from queue import Queue

ITER_NUM = 620
batch_size = 10

gan.generator.compile(
    loss=sm.losses.BinaryFocalLoss(),
    optimizer=Nadam(gan.generator_learning_rate),
    metrics=["accuracy"],
)

def batch_setter(queue):
    batch_i = 0
    count = 0
    while batch_i + gan.batch_size <= gan.data_loader.train_data_length and count < ITER_NUM:
        
        batch_index = gan.train_loaded_data_index[batch_i: batch_i +
                                                   gan.batch_size]        
        
        batch_tuple = gan.data_loader.get_data(
        data_mode="train", index=batch_index)

        queue.put(batch_tuple)
        queue.join()
        count += 1
    
def batch_getter(queue):
    
    original_img, masked_img = queue.get()
    tensor_original_img = tf.convert_to_tensor(original_img)
    tensor_masked_img = tf.convert_to_tensor(masked_img)
    queue.task_done()
    
    return tensor_original_img, tensor_masked_img
    
def batch_trainer(original_img, masked_img):
    
    gan.generator.train_on_batch(temp_source, temp_mask)

q = Queue()

setter = threading.Thread(target=batch_setter, args=(q,),daemon=True)
setter.start()
start_time = time.time()
for i in range(ITER_NUM):
    tensor_original_img, tensor_masked_img = batch_getter(q)
    
    gan.generator.train_on_batch(tensor_original_img, tensor_masked_img)
print(f"elapsed time : {time.time() - start_time}")

In [ ]:
start_time = time.time()
batch_i = 0
count = 0
while batch_i + gan.batch_size <= gan.data_loader.train_data_length and count < ITER_NUM:

    batch_index = gan.train_loaded_data_index[batch_i: batch_i +
                                               gan.batch_size]        
    batch_tuple = gan.data_loader.get_data(
    data_mode="train", index=batch_index)
    
    gan.generator.train_on_batch(*batch_tuple)
    
    count += 1
print(f"elapsed time : {time.time() - start_time}")

In [ ]:
import cv2

temp = tensor_masked_img
print(type(temp))
print(type(temp.numpy()))

In [ ]:
isinstance(temp.numpy(), tf.Tensor)

In [ ]:
from gan_module.model.build_model import build_dual_discriminator

temp = build_dual_discriminator(
            input_img_shape=(512,512,3),
            output_img_shape=(512,512,1),
            discriminator_power=1,
)

In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import Nadam

temp.compile(
    loss=[
        tf.keras.losses.BinaryCrossentropy(label_smoothing=0.1),
        tf.keras.losses.BinaryCrossentropy(label_smoothing=0.1)
    ],
    optimizer=Nadam(),
    metrics=["accuracy"],
)

In [ ]:
import numpy as np

image_mockup = np.ones((1,512,512,3))
mask_mockup = np.ones((1,512,512,1))
patch_mockup = np.ones((1,8,8,1))

temp.test_on_batch([image_mockup, mask_mockup], [patch_mockup, patch_mockup])